In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Creating a dataframe that has the following columns:
# 1. playID
# 2. nflID of tackler who made tackle for each playID
# 3. name of tackler who made tackle for each playID
# 4. Ball-carrier for each playID


#Import Data
games = pd.read_csv('../games.csv')
playerData = pd.read_csv('../players.csv')
tackleData = pd.read_csv('../tackles.csv')
#tracking1 = pd.read_csv('../tracking_week_1data.csv')


li = []

for i in range(9):
    string = '../tracking_week_' + str(i + 1) + 'data.csv'
    track = pd.read_csv(string)
    li.append(track)
    
trackingdata = pd.concat(li, axis=0, ignore_index=True)

plays = pd.read_csv("../plays.csv")


In [74]:
#Concrete Stats for each tackle
ball_carrier = pd.DataFrame(plays.loc[:, ['playId', 'ballCarrierId', 'gameId', 'down', 'yardsToGo', 'playResult']])
ball_carrier['timeLeft'] = ((4 - plays['quarter']) * 15 + plays['gameClock'].str[:-3].astype(int)).astype(str) + plays['gameClock'].str[-3:]
ball_carrier['scoreDif'] = plays['preSnapHomeScore'].sub(plays['preSnapVisitorScore'])
ball_carrier['endZoneDistance'] = 100 - plays['absoluteYardlineNumber'] - plays['playResult']
ball_carrier['yardsto1st'] = plays['yardsToGo'] - plays['playResult']
tackler = pd.DataFrame(tackleData.loc[:, ['playId', 'nflId', 'gameId']])
name = pd.DataFrame(playerData.loc[:, ['nflId', 'displayName']])

df = pd.merge(ball_carrier, tackler, on=('playId', 'gameId'))
df = pd.merge(df, name, on='nflId')
print(df)

       playId  ballCarrierId      gameId  down  yardsToGo  playResult  \
0        3537          48723  2022100908     1         10           9   
1          84          47853  2022110609     2         10           5   
2        1016          53555  2022102301     1         10           9   
3         879          52460  2022110609     3          1           2   
4          97          46155  2022102700     1         10           5   
...       ...            ...         ...   ...        ...         ...   
17421    3790          54556  2022101607     4          1           2   
17422    1945          53549  2022100905     2         10          -1   
17423    3201          47807  2022110604     2         10           3   
17424     779          53454  2022103000     2          2           2   
17425    2514          47911  2022100912     1         10           6   

      timeLeft  scoreDif  endZoneDistance  yardsto1st  nflId  \
0         7:52        14               22           1  4330

In [62]:
#Season tackling stats of each player
playerData = pd.read_csv("../players.csv")
tackleData = pd.read_csv("../tackles.csv")

tackles = {}

for row in range(len(playerData)):
    tackles[playerData.loc[row][0]] = [playerData.loc[row][0], 0, 0, 0, 0]


for row in range(len(tackleData)):
    if tackleData.loc[row][3] == 1:
        tackles[tackleData.loc[row][2]][1] = (tackles[tackleData.loc[row][2]][1] + 1)
    if tackleData.loc[row][4] == 1:
        tackles[tackleData.loc[row][2]][2] = (tackles[tackleData.loc[row][2]][2] + 1)
    if tackleData.loc[row][5] == 1:
        tackles[tackleData.loc[row][2]][3] = (tackles[tackleData.loc[row][2]][3] + 1)
    if tackleData.loc[row][6] == 1:
        tackles[tackleData.loc[row][2]][4] = (tackles[tackleData.loc[row][2]][4] + 1)
newTackles = pd.DataFrame(tackles)
newTackles = newTackles.transpose()
newTackles.rename(columns = {0:'nflId', 1:'Tackles', 2:'Assists', 3:'Forced Fumbles', 4:'Missed Tackles'}, inplace = True)
#newTackles.head()

merged = pd.merge(playerData, newTackles, on='nflId')
print(merged)

      nflId height  weight   birthDate       collegeName position  \
0     25511    6-4     225  1977-08-03          Michigan       QB   
1     29550    6-4     328  1982-01-22          Arkansas        T   
2     29851    6-2     225  1983-12-02        California       QB   
3     30842    6-6     267  1984-05-19              UCLA       TE   
4     33084    6-4     217  1985-05-17    Boston College       QB   
...     ...    ...     ...         ...               ...      ...   
1678  55200    6-6     266         NaN           Indiana       DT   
1679  55212    6-0     230         NaN        Iowa State      ILB   
1680  55239    6-2     300         NaN      Pennsylvania       DT   
1681  55240    6-1     185         NaN           Buffalo       CB   
1682  55241    6-2     280         NaN  Coastal Carolina       DT   

           displayName  Tackles  Assists  Forced Fumbles  Missed Tackles  
0            Tom Brady        0        0               0               0  
1         Jason Peter

In [75]:
#Distance between players for each tackle
outcomes = ['pass_outcome_caught', 'handoff', 'run']
tacklesDist = pd.DataFrame(df.loc[:, ['gameId', 'playId', 'nflId','ballCarrierId']])

#Defense x and y values
defenseLocation = pd.merge(trackingdata, tacklesDist, on=('gameId', 'playId', 'nflId'))
defenseDistance = defenseLocation[defenseLocation['event'].isin(outcomes)]

#Offense x and y values
offenseLocation = pd.merge(trackingdata, tacklesDist, left_on=('gameId', 'playId', 'nflId'), right_on=('gameId', 'playId', 'ballCarrierId'))
offenseDistance = offenseLocation[offenseLocation['event'].isin(outcomes)]

#Finding Euclidean Distance
def calculate_distance(row):
    return np.sqrt((row['x_x'] - row['x_y'])**2 + (row['y_x'] - row['y_y'])**2)

#Combining them
totaldistance = pd.merge(defenseDistance, offenseDistance, on=('gameId', 'playId'))
totaldistance['distanceBtPlayers'] = totaldistance.apply(calculate_distance, axis=1)
totaldistance = pd.DataFrame(totaldistance.loc[:, ['gameId','playId','nflId','x_x', 'y_x', 'x_y', 'y_y', 'event_y', 'ballCarrierId_y', 'distanceBtPlayers']])

In [77]:
#Final Clutch Stats Dataframe
newStats = pd.merge(df, totaldistance[['gameId','playId','nflId','distanceBtPlayers']], on=('gameId', 'playId','nflId'))
newStats = newStats.drop_duplicates()
newStats = newStats.drop(['ballCarrierId', 'nflId', 'yardsToGo'], axis=1)
newStats = newStats.reset_index(drop=True)
newStats.head()

,playId,gameId,down,playResult,timeLeft,scoreDif,endZoneDistance,yardsto1st,displayName,distanceBtPlayers
0,3537,2022100908,1,9,7:52,14,22,1,Keanu Neal,5.929798
1,84,2022110609,2,5,59:50,0,12,5,Keanu Neal,10.935890
2,1016,2022102301,1,9,41:29,0,31,1,Keanu Neal,18.493312
3,879,2022110609,3,2,45:58,3,6,-1,Keanu Neal,4.460605
4,97,2022102700,1,5,59:18,0,42,5,Keanu Neal,7.735069
